In [3]:
import pandas as pd
import json
import re

#the location of your input files. CHANGE BEFORE RUNNING
INPUT_FILE_LOC = '/content/drive/MyDrive/ryder diag'


# Following is required for writing the output. CHANGE BEFORE RUNNING
OUTPUT_FILE_LOC = '/content/drive/MyDrive/ryder diag'



with open(f"{INPUT_FILE_LOC}/ilm_policies.json") as f:
  ilm_data = json.load(f)

with open(f"{INPUT_FILE_LOC}/indices.json") as f:
  indices_data = json.load(f)

def extract_num_days(str):

  pattern = re.compile(r"\d+")
  extr = pattern.search(str)
  if extr:
    return int(extr.group())
  else:
    return 0

def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True

index_df = pd.DataFrame(columns=['index_name', 'store', 'docs', 'node', 'prirep'])

for index in indices_data:
  index_df.loc[len(index_df)] = [index.get('index'),index.get('store'), index.get('docs'), index.get('node'), index.get('prirep')]

#index_df.to_csv()



df = pd.DataFrame(columns=['policy', 'index_name', 'hot', 'warm', 'cold', 'frozen'])



for item in ilm_data:
  ilm_data_row = ilm_data.get(item)
  hot = 0
  hot_max = 0
  warm = 0
  cold = 0
  frozen = 0
  delete = 0

  if ilm_data_row.get('policy').get('phases').get('hot'):
    #hot_str = ilm_data_row.get('policy').get('phases').get('hot').get('min_age')
    if keys_exists(ilm_data_row,'policy','phases', 'hot', 'actions','rollover','max_age'):
      hot_max_age_str = ilm_data_row.get('policy').get('phases').get('hot').get('actions').get('rollover').get('max_age')
      #hot = extract_num_days(hot_str)
      hot_max = extract_num_days(hot_max_age_str)

  if ilm_data_row.get('policy').get('phases').get('warm'):
    warm_str = ilm_data_row.get('policy').get('phases').get('warm').get('min_age')
    warm = extract_num_days(warm_str)


  if ilm_data_row.get('policy').get('phases').get('cold'):
    cold_str = ilm_data_row.get('policy').get('phases').get('cold').get('min_age')
    cold = extract_num_days(cold_str)


  if ilm_data_row.get('policy').get('phases').get('frozen'):
    frozen_str = ilm_data_row.get('policy').get('phases').get('frozen').get('min_age')
    frozen = extract_num_days(frozen_str)

  if ilm_data_row.get('policy').get('phases').get('delete'):
    delete_str = ilm_data_row.get('policy').get('phases').get('delete').get('min_age')
    delete = extract_num_days(delete_str)


  if warm > 0:
    hot = warm
  elif cold > 0:
    hot = cold
  elif frozen > 0:
    hot = frozen
  else:
    hot = hot_max

  if cold > 0:
    warm = cold - hot
  elif frozen > 0:
    warm = frozen - (cold+hot)

  if frozen > 0:
    frozen = delete - (hot+warm+cold)







  in_use_by_indices = ilm_data_row.get("in_use_by").get("indices")
  for index_name in in_use_by_indices:
    df.loc[len(df.index)] = [item,index_name,hot,warm,cold,frozen]


df[['hot','warm','cold','frozen']] = df[['hot','warm','cold','frozen']].apply(pd.to_numeric)


merged_pd = pd.merge(df,index_df,on='index_name')

merged_pd.to_csv(f"{OUTPUT_FILE_LOC}/merged_indices.csv")



In [4]:
GB = 1000000000
primary_df = merged_pd.loc[merged_pd['prirep'] == 'p']
#primary_df = merged_pd
primary_df[['store', 'docs']] = primary_df[['store', 'docs']].apply(pd.to_numeric)

grouped_df = primary_df.groupby(["policy","hot","warm","cold","frozen"]).agg({'index_name':'count', 'store': 'sum', 'docs' : 'sum'})

store_in_hot_bytes_list = []
store_in_warm_bytes_list = []
store_in_cold_bytes_list = []
store_in_frozen_bytes_list  = []
daily_ingest_bytes_list = []
store_in_gb_list = []

for item in grouped_df.iterrows():
  hot = item[0][1]
  warm = item[0][2]
  cold = item[0][3]
  frozen = item[0][4]
  store = item[1][1]

  hot_days_ratio = 0
  warm_days_ratio = 0
  cold_days_ratio = 0
  frozen_days_ratio = 0
  daily_ingest_bytes = 0
  store_in_hot_bytes = 0
  total_days = hot + warm + cold #+ frozen


  if total_days > 0:
    hot_days_ratio = hot / total_days
    warm_days_ratio = warm / total_days
    cold_days_ratio = cold / total_days
    frozen_days_ratio = frozen / total_days


  store_in_hot_bytes = round(((store * hot_days_ratio)/GB),4)
  store_in_warm_bytes = round(((store * warm_days_ratio) / GB),4)
  store_in_cold_bytes = round(((store * cold_days_ratio) / GB),4)
  store_in_frozen_bytes = round(((store * frozen_days_ratio) / GB),4)
  store_in_gb = round((store/GB),4)



  if(hot > 0):
    daily_ingest_bytes = round((store_in_hot_bytes / hot),4)

  store_in_hot_bytes_list.append(store_in_hot_bytes)
  store_in_warm_bytes_list.append(store_in_warm_bytes)
  store_in_cold_bytes_list.append(store_in_cold_bytes)
  store_in_frozen_bytes_list.append(store_in_frozen_bytes)
  daily_ingest_bytes_list.append(daily_ingest_bytes)
  store_in_gb_list.append(store_in_gb)

grouped_df.insert(len(grouped_df.columns),column="storage(gb)",value=store_in_gb_list)
grouped_df.insert(len(grouped_df.columns),column="hot(gb)",value=store_in_hot_bytes_list)
grouped_df.insert(len(grouped_df.columns),column="warm(gb)",value=store_in_warm_bytes_list)
grouped_df.insert(len(grouped_df.columns),column="cold(gb)",value=store_in_cold_bytes_list)
grouped_df.insert(len(grouped_df.columns),column="frozen(gb)",value=store_in_frozen_bytes_list)
grouped_df.insert(len(grouped_df.columns),column="daily_ingest(gb)",value=daily_ingest_bytes_list)
grouped_df.drop(columns=["store"], inplace=True)
grouped_df.rename(columns={"index_name" : "indices"},inplace=True)

grouped_df.to_csv(f"{OUTPUT_FILE_LOC}/merged_ilm.csv")


grouped_df

<ipython-input-4-709cb108cbd3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_df[['store', 'docs']] = primary_df[['store', 'docs']].apply(pd.to_numeric)


,,,,,indices,docs,storage(gb),hot(gb),warm(gb),cold(gb),frozen(gb),daily_ingest(gb)
policy,hot,warm,cold,frozen,,,,,,,,
.alerts-ilm-policy,30,0,0,0,31,477094,2.9336,2.9336,0.0000,0.0,0.0000,0.0978
.deprecation-indexing-ilm-policy,30,0,0,0,2,161,0.0008,0.0008,0.0000,0.0,0.0000,0.0000
.fleet-actions-results-ilm-policy,30,0,0,0,3,40,0.0002,0.0002,0.0000,0.0,0.0000,0.0000
.items-security-operations-center,0,0,0,0,1,0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000
.lists-security-operations-center,0,0,0,0,1,0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000
filebeat,2,88,0,260,2519,171974469937,55002.6490,1222.2811,53780.3679,0.0,158896.5416,611.1406
ilm-history-ilm-policy,30,0,0,0,4,74816,0.0246,0.0246,0.0000,0.0,0.0000,0.0008
kibana-event-log-policy,30,0,0,0,14,11999992,2.0304,2.0304,0.0000,0.0,0.0000,0.0677
kibana-reporting,0,0,0,0,45,170,0.1164,0.0000,0.0000,0.0,0.0000,0.0000


In [ ]:
primary_df.dtypes